# Content Based Filtering by hand


**Learning Objectives**

1. Create and compute a user feature matrix.
2. Compute where each user lies in the feature embedding space.
3. Create recommendations for new movies based on similarity measures between the user and movie feature vectors.

## Introduction

This notebook illustrates how to implement a content based filter using low level Tensorflow operations.  
The code here follows the technique explained in Module 2 of Recommendation Engines: Content Based Filtering.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/content_based_by_hand.ipynb).

To run this lab, we need to use TensorFlow version 2.X. 

In [1]:
!pip3 install tensorflow

**Make sure to restart your kernel to ensure this change has taken place**.

In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.6.3


To start, we'll create our list of users, movies and features. While the users and movies represent elements in our database, for a content-based filtering method the features of the movies are likely hand-engineered and rely on domain knowledge to provide the best embedding space. Here we use the categories of Action, Sci-Fi, Comedy, Cartoon, and Drama to describe our movies (and thus our users).

In this example, we will assume our database consists of four users and six movies, listed below.  

In [2]:
users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = [
    'Star Wars', 'The Dark Knight', 'Shrek',
    'The Incredibles', 'Bleu', 'Memento'
]
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

### Initialize our users, movie ratings, and features

We'll need to enter the user's movie ratings and the k-hot encoded movie features matrix. Each row of the users_movies matrix represents a single user's rating (from 1 to 10) for each movie. A zero indicates that the user has not seen/rated that movie. The movies_feats matrix contains the features for each of the given movies. Each row represents one of the six movies, while the columns represent the five categories. A one indicates that a movie fits within a given genre/category. 

In [3]:
# Each row represents a user's rating for the different movies.
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]], dtype=tf.float32)

# Features of the movies one-hot encoded.
# e.g. columns could represent
# ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]], dtype=tf.float32)

2022-04-27 11:05:45.536583: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Computing the user feature matrix

We will compute the user feature matrix; that is, a matrix containing each user's embedding in the five-dimensional feature space.

**TODO 1**: Calculuate this as the matrix multiplication of the `users_movies` tensor with the `movies_feats` tensor.

In [4]:
users_feats = # TODO: Use matrix multplication to find the user features.
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

Next we normalize each user feature vector to sum to 1. Normalizing isn't strictly neccesary, but it makes it so that rating magnitudes will be comparable between users.

In [5]:
users_feats = users_feats / tf.reduce_sum(users_feats, axis=1, keepdims=True)
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

#### Ranking feature relevance for each user

We can use the users_feats computed above to represent the relative importance of each movie category for each user. 

In [6]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [7]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))

Ryan: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']


### Determining movie recommendations. 

We'll now use the `users_feats` tensor we computed above to determine the movie ratings and recommendations for each user.

To compute the projected ratings for each movie, we compute the similarity measure between the user's feature vector and the corresponding movie feature vector.  

We will use the dot product as our similarity measure. In essence, this is a weighted movie average for each user.

**TODO 2**: Implement this as a matrix multiplication. *Hint*: one of the operands will need to be transposed.

In [8]:
users_ratings = # TODO: Use matrix multplication to find user ratings.
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

The computation above finds the similarity measure between each user and each movie in our database. To focus only on the ratings for new movies, we apply a mask to the all_users_ratings matrix.  

If a user has already rated a movie, we ignore that rating. This way, we only focus on ratings for previously unseen/unrated movies.

In [9]:
users_unseen_movies = tf.equal(users_movies, tf.zeros_like(users_movies))
ignore_matrix = tf.zeros_like(tf.cast(users_movies, tf.float32))

users_ratings_new = tf.where(
    users_unseen_movies,
    users_ratings,
    ignore_matrix)

users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

Finally, let's grab and print out the top 2 rated movies for each user.

In [10]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

In [11]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i], movie_names))

Ryan: ['The Incredibles', 'Memento']
Danielle: ['The Incredibles', 'Star Wars']
Vijay: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']


Copyright 2022 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.